In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
%qtconsole
import sys
import collections
import itertools
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sys.path.append('../src/')
import data_processing
import ripple_decoding
import ripple_detection

In [4]:
Animal = collections.namedtuple('Animal', {'directory', 'short_name'})
num_days = 8
days = range(1, num_days + 1)
animals = {'HPc': Animal(directory='HPc_direct', short_name='HPc')}

epoch_info = data_processing.make_epochs_dataframe(animals, days)
tetrode_info = data_processing.make_tetrode_dataframe(animals)
epoch_index = (epoch_info
    .loc[(['HPc'], [8]), :]
    .loc[epoch_info.environment == 'wtr1'].index)[0]

cur_tetrode_info = tetrode_info[epoch_index]
cur_tetrode_info

area  depth descrip  numcells tetrode_id
animal day epoch_ind tetrode_number                                         
HPc    8   2         1               CA1    108  riptet         0     HPc821
                     2               CA1    100  riptet         1     HPc822
                     3               CA1     91  riptet         5     HPc823
                     4               CA1    104  riptet         0     HPc824
                     5               CA1     96  riptet         0     HPc825
                     6               CA1    106  riptet         3     HPc826
                     7               CA1     84  CA1Ref         0     HPc827
                     8               PFC     65     NaN         0     HPc828
                     9               PFC     60     NaN         0     HPc829
                     10              PFC     57     NaN         0    HPc8210
                     11              PFC     98     NaN         0    HPc8211
                     12              PFC    102     NaN         0    HPc8212
                     13              PFC     45  PFCRef         0    HPc8213
                     14              PFC     66     NaN         0    HPc8214
                     15              PFC     71     NaN         7    HPc8215
                     16              PFC    104     NaN         1    HPc8216
                     17              Ctx    282  CtxRef         0    HPc8217
                     18              Ctx    315     NaN         0    HPc8218
                     19              Ctx    318     NaN         0    HPc8219
                     20              Ctx    300     NaN         0    HPc8220
                     21              Ctx    196     NaN         0    HPc8221
                     22              Ctx      0     NaN         0    HPc8222

In [22]:
ripple_times = ripple_detection.get_epoch_ripples(
    epoch_index, animals, sampling_frequency=1500,
    ripple_detection_function=ripple_detection.Kay_method)
ripple_info, decision_state_probability, posterior_density, decision_state_names = \
    ripple_decoding.decode_ripple(epoch_index, animals, ripple_times)
ripple_info


Detecting ripples for Animal HPc, Day 8, Epoch #2:


Decoding ripples for Animal HPc, Day 8, Epoch #2:

	Fitting encoding model...


	Fitting state transition model...



/Users/edeno/anaconda3/envs/Jadhav-2016-Data-Analysis/lib/python3.5/site-packages/numpy/lib/function_base.py:804: RuntimeWarning: invalid value encountered in greater_equal
  not_smaller_than_edge = (sample[:, i] >= edges[i][-1])


	Setting initial conditions...

	Decoding ripples...



,ripple_type,ripple_state_probability,ripple_start_time,ripple_end_time,number_of_unique_neurons_spiking,number_of_spikes
ripple_number,,,,,,
1,inbound_forward,0.310823,11470.856367,11470.924367,4,13
2,outbound_reverse,0.293368,11473.335033,11473.423033,4,11
3,inbound_reverse,0.353773,11473.453700,11473.490367,2,5
4,inbound_forward,0.298346,11486.904367,11486.947700,4,9
5,outbound_forward,0.326074,11489.198367,11489.245033,1,1
6,outbound_forward,0.327860,11490.705033,11490.755700,1,1
7,outbound_forward,0.310067,11491.035700,11491.090367,3,4
8,outbound_reverse,0.324401,11492.161700,11492.257700,4,9
9,outbound_reverse,0.353120,11492.565033,11492.613700,2,2


In [ ]:
lfps = list(data_processing.get_LFP_data(cur_tetrode_info.index, animals).values())
for ripples in ripple_info.groupby('ripple_type'):
    ripple_times_by_type = ripples.loc[:, ('ripple_start_time', 'ripple_end_time')].values.tolist()
    coherence = [spectral.difference_from_baseline_coherence(
                [lfp1, lfp2], ripple_times_by_type, **params)
                     for lfp1, lfp2 in itertools.combinations(lfps, 2)]